# Exercise 3.2

In [7]:
import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import pickle
from collections import OrderedDict
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [8]:
# Define the model
case = 'b'
num_classes = 10
num_epochs = 500

In [9]:
# 3. Define model for case 'a'
case = 'a'
num_classes = 10
num_epochs = 500

if case == 'a':
    inputs, n_hidden0, n_hidden1, out = 784*3, 64, 16, 10
    ckpt_pth = 'best_model_NN.pth'
    model = nn.Sequential(
        nn.Linear(inputs, n_hidden0, bias=True),
        nn.Tanh(),
        nn.Linear(n_hidden0, n_hidden1, bias=True),
        nn.Tanh(),
        nn.Linear(n_hidden1, out, bias=True),
        nn.Softmax(dim=1)
    ).to('cuda')
elif case == 'b':
    ckpt_pth = 'best_model_CNN.pth'
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
    model.classifier[6] = nn.Linear(4096, num_classes)
    model = model.to('cuda')
else:
    raise ValueError('Case choice is invalid')

model.train()

#load data
dev_path = './data/0_development_data.pkl'
test_path = './data/0_test_data.pkl'

with open(dev_path, 'rb') as f:
    devel_data = pickle.load(f)

with open(test_path, 'rb') as f:
    test_data = pickle.load(f)

#combine
combined_imgs = devel_data[0] + test_data[0]
combined_labels = [int(i[0].split('/')[-2]) for i in combined_imgs]

#split train and test
train_imgs, temp_imgs, train_labels, temp_labels = train_test_split(
    combined_imgs, combined_labels, test_size=0.25, stratify=combined_labels, random_state=42)

#Split train and val
val_imgs, test_imgs, val_labels, test_labels = train_test_split(
    temp_imgs, temp_labels, test_size=0.4, stratify=temp_labels, random_state=42)

#check counts
print(f"Train: {len(train_imgs)}, Validation: {len(val_imgs)}, Test: {len(test_imgs)}")

Train: 31500, Validation: 6300, Test: 4200


In [10]:

class CustomDataset(Dataset):
    def __init__(self, image_list, labels, transform=None):
        self.image_list = image_list
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        if case == 'a':
            image = self.image_list[idx].astype(float)
            image /= 255.0
            image -= np.sum(np.sum(image, 0), 0) / (image.shape[0] * image.shape[1])
        elif case == 'b':
            img_tmp = self.image_list[idx]
            image = preprocess(Image.fromarray(img_tmp))
        label = self.labels[idx]
        return image, label

criterion = nn.CrossEntropyLoss()
if case == 'a':
    optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
else:
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

train_array_list = [i[1] for i in train_imgs]
val_array_list = [i[1] for i in val_imgs]
test_array_list = [i[1] for i in test_imgs]

dataset_train = CustomDataset(train_array_list, train_labels, transform=None)
dataset_val = CustomDataset(val_array_list, val_labels, transform=None)
dataset_test = CustomDataset(test_array_list, test_labels, transform=None)

batch_size = 32
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [11]:
# Training loop
early_stopping_patience = 10

best_val_loss = 10000.0
for epoch in range(num_epochs):
    running_loss, running_val_loss = 0.0, 0.0
    model.train()
    for inputs_, labels_ in tqdm(dataloader_train):

        if case == 'a': inputs_ = torch.reshape(inputs_, (inputs_.shape[0], -1))
        inputs_, labels_ = inputs_.to(torch.float).to('cuda'), labels_.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs_)
        loss = criterion(outputs, labels_)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    with torch.no_grad():
        for inputs_val, labels_val in tqdm(dataloader_val):
            if case == 'a': inputs_val = torch.reshape(inputs_val, (inputs_val.shape[0], -1))
            inputs_val, labels_val = inputs_val.to(torch.float).to('cuda'), labels_val.to('cuda')
            outputs_val = model(inputs_val)
            val_loss = criterion(outputs_val, labels_val)
            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss/len(dataloader_val)
    if epoch_val_loss < best_val_loss:
        early_stopping_counter = 0
        best_val_loss = float(epoch_val_loss)
        torch.save(model.state_dict(), ckpt_pth)
    else:
        early_stopping_counter += 1
        if early_stopping_counter==early_stopping_patience:
            print('-------- Early Stopping ------------')
            print(f'Epoch {epoch+1}, Train loss: {running_loss/len(dataloader_train)}, Val loss: {running_val_loss/len(dataloader_val)}')
            break

    print(f'Epoch {epoch+1}, Train loss: {running_loss/len(dataloader_train)}, Val loss: {epoch_val_loss}')

100%|██████████| 197/197 [00:00<00:00, 574.65it/s]


Epoch 1, Train loss: 1.622951765713958, Val loss: 1.5112204013137043


100%|██████████| 197/197 [00:00<00:00, 565.81it/s]


Epoch 2, Train loss: 1.49711089775647, Val loss: 1.4934296390126804


100%|██████████| 197/197 [00:00<00:00, 510.62it/s]


...

Epoch 36, Train loss: 1.463599018397065, Val loss: 1.474071706612098


100%|██████████| 197/197 [00:00<00:00, 569.53it/s]


Epoch 37, Train loss: 1.4627684539949835, Val loss: 1.4735093177272585


100%|██████████| 197/197 [00:00<00:00, 570.49it/s]

-------- Early Stopping ------------
Epoch 38, Train loss: 1.463510664344439, Val loss: 1.4752131282980672


In [12]:
# Make predictions on the train data
model.load_state_dict(torch.load(ckpt_pth, weights_only=True))
model.eval()

Sequential(
  (0): Linear(in_features=2352, out_features=64, bias=True)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=16, bias=True)
  (3): Tanh()
  (4): Linear(in_features=16, out_features=10, bias=True)
  (5): Softmax(dim=1)
)

In [13]:
# Inference function
def get_predictions(input_batch, model):

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)

    probabilities = torch.nn.functional.softmax(output, dim=1)
    return probabilities

In [14]:
# Run inference
preds_list = []
batch_size = 64
with torch.no_grad():
    for inputs_test, _ in tqdm(dataloader_test):
        if case == 'a': inputs_test = torch.reshape(inputs_test, (inputs_test.shape[0], -1))
        inputs_test = inputs_test.to(torch.float).to('cuda')
        preds_list.append(get_predictions(inputs_test, model).cpu().numpy())
final_preds = np.argmax(np.reshape(np.vstack(preds_list), (-1,10)),1)

100%|██████████| 132/132 [00:00<00:00, 521.32it/s]


In [15]:
# Generate all interesting metrics
def multiclass_metrics(y_true, y_pred, labels):
    """
    Compute per-class accuracy, sensitivity (recall), specificity, and precision.

    y_true, y_pred : array-like of shape (n_samples,)
    labels         : list of class labels, e.g. [0,1,...,9]
    """
    # Compute the full confusion matrix once
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    # cm[i, j] is count of true class i predicted as class j

    # Prepare containers
    metrics = {
        "class": [],
        "accuracy": [],
        "sensitivity (recall)": [],
        "specificity": [],
        "precision": []
    }

    # Total samples
    total = cm.sum()

    for idx, cls in enumerate(labels):
        TP = cm[idx, idx]
        FN = cm[idx, :].sum() - TP
        FP = cm[:, idx].sum() - TP
        TN = total - TP - FP - FN

        # Per-class metrics
        acc = (TP + TN) / total
        sens = TP / (TP + FN) if (TP + FN) > 0 else 0.0
        spec = TN / (TN + FP) if (TN + FP) > 0 else 0.0
        prec = TP / (TP + FP) if (TP + FP) > 0 else 0.0

        metrics["class"].append(cls)
        metrics["accuracy"].append(acc)
        metrics["sensitivity (recall)"].append(sens)
        metrics["specificity"].append(spec)
        metrics["precision"].append(prec)

    return pd.DataFrame(metrics)


In [17]:
# Report the performance
report_df = multiclass_metrics(test_labels, final_preds, np.arange(10).tolist()).set_index('class')
report_df

,accuracy,sensitivity (recall),specificity,precision
class,,,,
0,0.999524,0.997579,0.999736,0.997579
1,0.999286,0.993590,1.000000,1.000000
2,0.997857,0.997608,0.997885,0.981176
3,0.997619,0.981609,0.999469,0.995338
4,0.999524,0.997543,0.999736,0.997543
5,0.999286,0.997368,0.999476,0.994751
6,0.999048,0.992754,0.999736,0.997573
7,0.997143,0.984091,0.998670,0.988584
8,0.995952,0.985222,0.997101,0.973236


In [18]:
# 1. Save the DataFrame to a CSV file
report_df.to_csv("classification_report_improved_caseb.csv")

# 2. Download it from Colab to your computer
from google.colab import files
files.download("classification_report_improved_caseb.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>